In [455]:
import numpy as np
from random import randint, shuffle, random

In [456]:
# Chargement de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(2, 10)
            if i == j:
                cout = 0
            i_after = i+1 if i+1 < n_ville else 0
            if i_after == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout
    mat[n_ville-1][0] = 1
    mat[0][n_ville-1] = 1

    return mat
 

In [457]:
#mat=[[0,2,1],[2,0,1],[1,2,0]]
#population=[]


# Création de la population initial, de taille size
def population_init(mat, size, n_lives=5):
    population = []
    
    for _ in range(0, size):
        circuit = list(range(0, len(mat[0])))
        shuffle(circuit) # circuit random
        population.append([circuit, n_lives])
        
    return population

#population_init(mat, 5)

In [458]:
#mat = [ [0,5,2], [4,0,3], [1,5,0] ]

def fitness(path, cost_mat): # Renvoie le coût d'un chemin, par rapport à la matrice d'adjacence
    travel_length = 0
    
    for i in range(0, len(path)):
        depart = path[i-1] # On compte aussi l'arrète entre la premiere et dernière ville (-1 en python -> dernier élément) 
        arrival = path[i]
        travel_length += cost_mat[depart][arrival]
    
    return travel_length  

#fitness(mat, [1,0,2])

In [459]:
def mort(pop):
    # Décremente la vie d'un individu et tue si <= 0
    new_pop = []
    
    for path in pop:
        path[1] -= 1
        
        # On ne garde que les individus vivants
        if path[1] >= 0:
            new_pop.append(path)
            
    return new_pop

In [460]:
def selection(population, mat_cost): 
    sum = 0
    # Somme des fitness pour ponderation
    for path in population:
        sum += fitness(path[0], mat_cost) 
    
    # Selection des individus de la population suivant un random
    new_pop = []
    for path in population:
        f = fitness(path[0], mat_cost)
        # Plus le fitness est bas, plus l'individu a de chance d'être selectionné
        if random() * sum > f:
            new_pop.append(path)
            
    return new_pop
    
        


In [461]:
def croisement(population, nb_enfants, life_time):
    for _ in range(0, nb_enfants // 2):
        # Selection des parents
        indice_parent1 = randint(0, len(population)-1)
        indice_parent2 = randint(0, len(population)-1)
        #print("indice du parent1:",indice_parent1,"  indice du parent 2:",indice_parent2)
        #print("taille de la pop:",len(population)-1)

        parent1=population[indice_parent1][0]
        parent2=population[indice_parent2][0]
        indice_same=randint(1, len(parent1) - 1)
        enfant1 = []
        enfant2 = []
        enfant1_temp = []
        enfant2_temp = []

        for x in range(0, indice_same):
            enfant1_temp.append(parent1[x])
            enfant2_temp.append(parent2[x])

        for y in range(indice_same, len(parent1)):
            enfant1_temp.append(parent2[y])
            enfant2_temp.append(parent1[y])

        for a in range(0, len(enfant1_temp)):
            if enfant1_temp[a] not in enfant1:
                enfant1.append(enfant1_temp[a])
            if enfant2_temp[a] not in enfant2:
                enfant2.append(enfant2_temp[a])
        
        for b in range(0, len(parent1)):
            if b not in enfant1:
                enfant1.append(b)
            if b not in enfant2:
                enfant2.append(b)
        
        population.append([enfant1, life_time])
        population.append([enfant2, life_time])
        #print(population)

#pop=[[[0,1,2,3],5],[[1,2,3,0],5]]
#croisement_v2(pop)
#print(pop)

In [462]:
def mutation(pop):
    mut=[]
    while(len(mut) != 3):
        pos = randint(0, len(pop)-1)
        mut.append(pos)
        
    for i in mut:
        perm1 = randint(0, len(pop[i][0])-1)
        perm2 = randint(0, len(pop[i][0])-1)
        #print(pop[i],"->",perm1,"   ",perm2)
        val1 = pop[i][0][perm1]
        val2 = pop[i][0][perm2]
        temp = val2
        pop[i][0][perm2]=val1
        pop[i][0][perm1]=temp
        #temp=pop[i][0][perm2]
        #perm2=pop[i][0][perm1]
        #perm1=temp
#pop=[[[0,1,2,3],5],[[4,5,6,7],5],[[8,9,1,2],5],[[3,4,5,6],5]]
#mutation(pop)
#print(pop)


def mutation(pop, mutation_amount, mutation_influence):  # Quantité de mutation et influence en pourcentage
    last_index = len(pop) - 1
    n_ville = len(pop[0][0])
    # Tant qu'on a des mutation à effectuer
    while last_index >= len(pop) - mutation_amount * len(pop) :
        # Random entre seulement les parties de la population qui n'ont pas déjà recu de mutation
        i = randint(0, last_index) 
        # On applique la mutation
        for _ in range(int(mutation_influence * n_ville)):
            curr_indiv = pop[i][0]
            r_index_1 = randint(0, n_ville-1)
            r_index_2 = randint(0, n_ville-1)
            while r_index_2 == r_index_1: # forcer une autre valeur
                r_index_2 = randint(0, n_ville-1)
            # Swap de deux valeurs randoms
            curr_indiv[r_index_1], curr_indiv[r_index_2] = curr_indiv[r_index_2], curr_indiv[r_index_1]
            
        # On deplace l'individu muté vers la partie de la population déjà muté
        pop[i], pop[last_index] = pop[last_index], pop[i]
        last_index -= 1

In [463]:
def verif(population, mat_cost):
    valeur = float('inf')
    
    for indiv in population:
        f = fitness(indiv[0], mat_cost)
        if f <= valeur:
            valeur = f
            chemin = indiv[0]
    return chemin, valeur
#mat=[[0,3,4,2],[5,0,6,1],[4,6,0,3],[2,1,3,0]]
#pop=[[[0,1,2,3],5],[[3,1,2,0],5],[[0,3,1,2],5],[[3,2,1,0],5]]
#val,chemin=verif(pop,999)
#print(val,"  ",chemin)

In [464]:
def algo_genetique(mat_cost, verbal=False):  
    ###### Paramètres ######
    time_max = 100 # Nombre de cycle max
    population_size = 10 # Taille initial de la population
    life_time = 5 # Nombre de cycle avant la mort d'un individu
    nb_enfants = 6 # Nombre d'enfant par cycle
    # Pourcentage de mutation
    mutation_amount = 0.2 # Taux d'individu a muter dans la population
    mutation_influence = 0.8 # Taux de changement sur l'individu a muter
    
    if verbal:
        print(mat_cost)
    
    time = 0 # Cycle actuel
    val_min = float("inf")
    population = population_init(mat_cost, population_size, life_time) 
    population = selection(population, mat_cost)
     
    while time < time_max and len(population) >= 2: 
        croisement(population, nb_enfants, life_time)
        mutation(population, mutation_amount, mutation_influence)
        population = selection(population, mat_cost)
        population = mort(population) 
        time += 1 
        
        if verbal:
            chemin_min, val_min = verif(population, mat_cost) 
            print("iteration:", time, ", nb individu:", len(population), "val_min :", val_min ) 
            
    
    chemin_min, val_min = verif(population, mat_cost) 
    if verbal: 
        print("Chemin min: ", chemin_min, " pour un cout de: ", val_min)
    return chemin_min, val_min
    
algo_genetique(create_matrix(10), verbal=True)

[[ 0.  1.  3.  6.  6.  3.  5.  4.  8.  1.]
 [ 1.  0.  1.  5.  6.  7.  7.  4.  3.  7.]
 [ 3.  1.  0.  1. 10.  9.  5. 10. 10.  9.]
 [ 6.  5.  1.  0.  1.  9.  2.  3. 10.  8.]
 [ 6.  6. 10.  1.  0.  1.  2.  3.  4.  2.]
 [ 3.  7.  9.  9.  1.  0.  1.  5.  3.  7.]
 [ 5.  7.  5.  2.  2.  1.  0.  1.  3.  2.]
 [ 4.  4. 10.  3.  3.  5.  1.  0.  1.  7.]
 [ 8.  3. 10. 10.  4.  3.  3.  1.  0.  1.]
 [ 1.  7.  9.  8.  2.  7.  2.  7.  1.  0.]]
iteration: 0 , nb individu: 10 val_min : 29.0
iteration: 1 , nb individu: 14 val_min : 28.0
iteration: 2 , nb individu: 19 val_min : 29.0
iteration: 3 , nb individu: 22 val_min : 29.0
iteration: 4 , nb individu: 26 val_min : 31.0
iteration: 5 , nb individu: 31 val_min : 32.0
iteration: 6 , nb individu: 27 val_min : 32.0
iteration: 7 , nb individu: 27 val_min : 34.0
iteration: 8 , nb individu: 26 val_min : 26.0
iteration: 9 , nb individu: 26 val_min : 26.0
iteration: 10 , nb individu: 26 val_min : 29.0
iteration: 11 , nb individu: 26 val_min : 30.0
iteration: 12 ,

([9, 1, 3, 2, 0, 7, 4, 6, 5, 8], 30.0)